<h2>Information Retrieval Assignment 1</h2>
<h4>Group ID: 26</h4>
<h4>Group Members Name with Student ID:</h4>
<h4>1. KARTHIKEYAN J - 2024AA05372</h4>
<h4>2. JANGALE SAVEDANA SUBHASH PRATIBHA - 2024AA05187</h4>
<h4>3. GANAPATHY SUBRAMANIAN S - 2024AA05188</h4>
<h4>4. ANANDAN A - 2024AA05269</h4>

<h3>Problem Statement</h3>
<h4>Designing a Text Search and Query Correction System using Levenshtein Edit Distance algorithm for Medical Documents</h4`>

# 1. Import and download the required libraries

In [4]:
import re
import os
from collections import defaultdict
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import PyPDF2
import pandas as pd
from docx import Document  # Must be imported!
import os
from IPython.display import display, Markdown

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\saved\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\saved\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\saved\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Global variables and NLP setup

In [6]:
# Global variables
inverted_index = defaultdict(set)
all_terms = set()
documents = []
doc_metadata = []

# NLP setup
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

## 1. Data Preprocessing
    a) Load documents from the directory provided.
    b) Preprocess each document. Will display terms, unique terms and sample terms from each document.Remove all punctuation, numbers, and special    characters from the dataset. 
## 2. create non-positional inverted index in descending order.

In [8]:
#  User defined functions to remove all punctuation, numbers, and special characters from the dataset. 
#  Apply lemmatization techniques to convert words to their base or root forms.
def preprocess_text(text):
    """Full preprocessing with intermediate steps"""
    print("\n=== ORIGINAL TEXT (SAMPLE) ===")
    print(text[:200] + "...\n" if len(text) > 200 else text)
    
    # 1. Clean text
    cleaned = re.sub(r'[^a-zA-Z0-9\s]', '', text.lower())
    print("=== AFTER CLEANING ===")
    print(cleaned[:200] + "...\n" if len(cleaned) > 200 else cleaned)
    
    # 2. Tokenization
    tokens = word_tokenize(cleaned)
    print(f"TOKENS ({len(tokens)}):", tokens[:30], "...\n")
    
    # 3. Stopword removal
    filtered = [w for w in tokens if w not in stop_words and len(w) > 2]
    print(f"AFTER STOPWORD REMOVAL ({len(filtered)}):", filtered[:30], "...\n")
    
    # 4. Lemmatization
    lemmatized = [lemmatizer.lemmatize(w) for w in filtered]
    print(f"FINAL PROCESSED TERMS ({len(lemmatized)}):", lemmatized[:30], "...")
    
    return lemmatized

In [9]:
# User defined functions to read different types of files from a directory.
def read_txt(file_path):
    """Read text file"""
    encodings = ['utf-8', 'latin-1', 'windows-1252']
    for encoding in encodings:
        with open(file_path, 'r', encoding=encoding) as f:
                return f.read()
        
def read_pdf(file_path):
    """Read PDF file"""
    text = ""
    with open(file_path, 'rb') as file:
            reader = PyPDF2.PdfReader(file)
            for page in reader.pages:
                text += page.extract_text()
    return text

def read_csv(file_path):
    """Read CSV file"""
    encodings = ['utf-8', 'latin-1', 'windows-1252']
    for encoding in encodings:
       df = pd.read_csv(file_path, encoding=encoding)
       return ' '.join(df.select_dtypes(include=['object']).astype(str).values.flatten())

def read_excel(file_path):
    """Read Excel file"""
    df = pd.read_excel(file_path)
    return ' '.join(df.select_dtypes(include=['object']).astype(str).values.flatten())

def read_docx(file_path):
    """Read Word DOCX file"""
    doc = Document(file_path)
    return '\n'.join([para.text for para in doc.paragraphs])


In [10]:
def load_documents(directory):
    """Load documents from directory and build index"""
    global documents, doc_metadata, inverted_index, all_terms
    document_metadata = []
    
    if not os.path.exists(directory):
        raise FileNotFoundError(f"Directory not found: {directory}")
    
    print(f"Loading documents from: {directory}")
    
    for root, _, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                if file.endswith('.txt'):
                    text = read_txt(file_path)
                elif file.endswith('.pdf'):
                    text = read_pdf(file_path)
                elif file.endswith('.csv'):
                    text = read_csv(file_path)
                elif file.endswith(('.xls', '.xlsx')):
                    text = read_excel(file_path)
                elif file.endswith('.docx'):
                    text = read_docx(file_path)
                else:
                    continue
                
                if text.strip():
                    doc_id = len(documents)
                    documents.append(text)
                    doc_metadata.append({
                        'file_name': file,
                        'file_path': file_path
                    })

                    print(f"\n===============Loading: {file}=====================================")
                    # Add to index
                    terms = preprocess_text(text)  # preprocessing each 
                    for term in terms:
                        inverted_index[term].add(doc_id) # inverted index creation
                        all_terms.add(term)

                     # Store metadata - PROPERLY INDENTED
                    document_metadata.append({
                        'doc_id': doc_id,
                        'filename': file,
                        'filetype': os.path.splitext(file)[1],
                        'terms': len(terms),
                        'unique_terms': len(set(terms))
                    })
            
                    # Display file processing info
                    print(f"\n - {file} ({document_metadata[-1]['filetype']})")
                    print(f"  - Total Terms: {document_metadata[-1]['terms']}")
                    print(f"  - Unique terms: {document_metadata[-1]['unique_terms']}")
                    print(f"  - Sample unique terms: {list(set(terms))[:5]}...")
                       
                    print(f"\n Loaded: {file}")
            except Exception as e:
                print(f"Error processing {file}: {str(e)}")

    print(f"\nTOTAL SUMMARY")
    print(f"\nTotal documents loaded: {len(documents)}")
    print(f"Unique terms in index: {len(all_terms)}")

  #2)   # **********************Show most frequent terms - Sorted index creation******************************
    top_terms = sorted(inverted_index.items(), 
                      key=lambda x: len(x[1]), 
                      reverse=True)[:5]

    print("\n**************Sorted index*****************")
    print("\nTop 5 terms:")
    for term, doc_ids in top_terms:
        print(f"  {term}: appears in {(doc_ids)} documents")

    return inverted_index, document_metadata

In [11]:
 # Load documents
directory = "D:/AIML/IR/Assignment/medical_documents/"
print(directory)
path = os.path.abspath(directory)
print(f"\n===============OUTPUT=============================")
inverted_index, document_metadata  =  load_documents(path) #load all the document from directory

D:/AIML/IR/Assignment/medical_documents/

===============OUTPUT=============================
Loading documents from: D:\AIML\IR\Assignment\medical_documents

===============Loading: Cardio.pdf=====================================

=== ORIGINAL TEXT (SAMPLE) ===
Cardiologist-Level Arrhythmia Detection with Convolutional Neural Networks
Pranav RajpurkarPRANAVSR @CS.STANFORD .EDU
Awni Y. HannunAWNI @CS.STANFORD .EDU
Masoumeh Haghpanahi MHAGHPANAHI @IRHYTHMTEC...

=== AFTER CLEANING ===
cardiologistlevel arrhythmia detection with convolutional neural networks
pranav rajpurkarpranavsr csstanford edu
awni y hannunawni csstanford edu
masoumeh haghpanahi mhaghpanahi irhythmtech com
codie...

TOKENS (4517): ['cardiologistlevel', 'arrhythmia', 'detection', 'with', 'convolutional', 'neural', 'networks', 'pranav', 'rajpurkarpranavsr', 'csstanford', 'edu', 'awni', 'y', 'hannunawni', 'csstanford', 'edu', 'masoumeh', 'haghpanahi', 'mhaghpanahi', 'irhythmtech', 'com', 'codie', 'bourn', 'cbourn', 'ir

**Justification:**

1) Above function first load all the documents with different file extensions from the given directory
2) Preprocessing is done for each document.
    a) ORIGINAL TEXT (SAMPLE) shows few line from the document
    b) AFTER CLEANING shows text after removing special charaters, then converting all text to lowercase
    c) TOKENS (stream of text converted into smaller units called tokens) are extracted from each document
    d) AFTER STOPWORD REMOVAL removes all the stopwords(common English words to be excluded from each document
    e) FINAL PROCESSED TERMS shows all the words after Lemmatization(reduce words to their base or dictionary form).

3) Sorted index is created after preprocessing step. All the indexed are sorted and top 5 terms are displayed appearing in respective documents

**Purpose:**

Preprocess text for improving search efficiency with efficient indexing, accuracy, and relevance.

### 3. Wildcard search and regular search

In [14]:
# Search Functions
def wildcard_search(query, inverted_index):
    if not query.endswith('*'):
        return []
    prefix = query[:-1].lower()
    return sorted([term for term in inverted_index.keys() 
                 if term.startswith(prefix)])

def regular_search(query, inverted_index, doc_metadata):
    terms = preprocess_text(query)
    if not terms:
        return []
    
    # Find documents containing ALL terms (AND logic)
    matching_docs = None
    for term in terms:
        if term in inverted_index:
            if matching_docs is None:
                matching_docs = set(inverted_index[term])
            else:
                matching_docs.intersection_update(inverted_index[term])
        else:
            return []  # If any term doesn't exist, return nothing
    
    return list(matching_docs) if matching_docs else []

def search(query, inverted_index, doc_metadata):
    if query.endswith('*'):
        terms = wildcard_search(query, inverted_index)
        # For wildcard searches, return terms with document counts
        enriched_terms = []
        for term in terms:
            doc_count = len(inverted_index.get(term, []))
            enriched_terms.append({
                'term': term,
                'doc_count': doc_count,
                'example_docs': list(inverted_index.get(term, []))[:3]  # Show first 3 docs
            })
        return {
            'type': 'wildcard',
            'query': query,
            'count': len(terms),
            'results': enriched_terms  
        }
    else:
        doc_ids = regular_search(query, inverted_index, doc_metadata)
        results = []
        for doc_id in doc_ids:
            doc = doc_metadata[doc_id]
            results.append({
                'doc_id': doc_id,
                'filename': doc['filename']
            })
        return {
            'type': 'regular', 
            'query': query,
            'count': len(results),
            'results': results
        }

In [35]:
print("\n***************TESTING SEARCHES**********************")
print("\nSearch options:")
print("- Regular search: 'eg: diabetes'")
print("- Wildcard search: 'eg: cardio*'")
print("Type 'exit' to quit\n")
         
while True:
   
    query = input("\nEnter Search term: ").strip()

    if query.lower() == 'exit':
        break
 
    results = search(query, inverted_index, document_metadata)
    
    if results['type'] == 'wildcard':
        # Wildcard search results
            if results['count'] > 0:
                for term_info in results['results'][:10]:
                    print(f"- {term_info['term']} (in {term_info['doc_count']} documents)")
            else:
                print("\nNo matching terms found")  # Wildcard-specific no-results message
    
    else:
            # Regular search results
            if results['count'] > 0:
                for doc in results['results'][:10]:
                    print(f"- Document: {doc['filename']}")
            else:
                print("\nNo direct matches found")  # ✅ Only shows when count == 0
   


***************TESTING SEARCHES**********************

Search options:
- Regular search: 'eg: diabetes'
- Wildcard search: 'eg: cardio*'
Type 'exit' to quit




Enter Search term:  cardio*


- cardio (in 2 documents)
- cardiogenic (in 1 documents)
- cardiographic (in 1 documents)
- cardiol (in 1 documents)
- cardiolo (in 1 documents)
- cardiologist (in 2 documents)
- cardiologistlevel (in 1 documents)
- cardiology (in 4 documents)
- cardiopulmonary (in 4 documents)
- cardiovascular (in 9 documents)



Enter Search term:  patient



=== ORIGINAL TEXT (SAMPLE) ===
patient
=== AFTER CLEANING ===
patient
TOKENS (1): ['patient'] ...

AFTER STOPWORD REMOVAL (1): ['patient'] ...

FINAL PROCESSED TERMS (1): ['patient'] ...
- Document: Cardio.pdf
- Document: Cardiovascular  Pulmonary.txt
- Document: DataAnalyticsinhealthcare.pdf
- Document: gender-differences-arteries.pdf
- Document: Medical Specialty.txt
- Document: Medical Specialty_Gastro.pdf
- Document: Medical_history.docx
- Document: mtsamples.csv
- Document: mtsamples.xlsx
- Document: train.txt



Enter Search term:  exit


### 4.  Levenshtein distance logic and suggest terms for misspelled search strings based on distance

In [17]:
def levenshtein(s1, s2):
    if len(s1) < len(s2):
        return levenshtein(s2, s1)
    
    if len(s2) == 0:
        return len(s1)
    
    prev_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        curr_row = [i + 1]
        for j, c2 in enumerate(s2):
            inserts = prev_row[j + 1] + 1
            deletes = curr_row[j] + 1
            substitute = prev_row[j] + (c1 != c2)
            curr_row.append(min(inserts, deletes, substitute))
        prev_row = curr_row
    
    return prev_row[-1]

def suggest_terms(misspelled_word, inverted_index, max_suggestions=5):
    # First check for quick matches with common errors
    suggestions = []
    
    # Calculate distances to all terms in our vocabulary
    distances = []
    for correct_word in inverted_index.keys():
        distance = levenshtein(misspelled_word.lower(), correct_word.lower())
        distances.append((correct_word, distance))
    
    # Sort by distance (closest first)
    distances.sort(key=lambda x: x[1])
    
    # Get the top N suggestions with smallest distance
    closest_matches = [word for word, dist in distances[:max_suggestions]]
    
    return closest_matches

**Justification:**

Above functions are Levenshtein distance calculation. The Levenshtein distance (or edit distance) measures the minimum number of single-character edits (insertions, deletions, or substitutions) required to transform one string into another. 

Second function suggest_terms Used to find near matches for misspelled terms using levenshtein function.

In [31]:
print("\n **********OUTPUT*********")
print("\nType Mispelled word: 'eg: cardeo'")
print("Type 'exit' to quit")
while True:
            query = input("\nEnter Search term: ").strip()
            
            if query.lower() == 'exit':
                break  # Exit the loop before processing
        
            print("\nDid you mean:")
            suggestions = suggest_terms(query, inverted_index)
            print(f"'{query}': {suggestions}")
    
            


 **********OUTPUT*********

Type Mispelled word: 'eg: cardeo'
Type 'exit' to quit



Enter Search term:  dybetes



Did you mean:
'dybetes': ['diabetes', 'detec', 'better', 'deep', 'detect']



Enter Search term:  exit


**Justification**
Above functions are created when user wants to search terms.

***Purpose of wildcard search:***
Enables prefix-based searching (e.g., "cardio*" finds "cardiovascular", "cardiology")
Supports exploratory searches when users know only the beginning of terms

***Purpose of regular search:***
Performs exact term matching with AND logic
Handles preprocessed queries (tokenized, normalized)